Challenge Set 1

Topic:        Explore MTA turnstile data

Date:         1/13/2016

Name:         Justin Stewart


# Challenge 1:

Download a few MTA turnstile data files. Open up a file, use csv reader to read it, make a python dict where there is a key for each (C/A, UNIT, SCP, STATION). These are the first four columns. The value for this key should be a list of lists. Each list in the list is the rest of the columns in a row.

In [ ]:
#!curl http://web.mta.info/developers/data/nyct/turnstile/turnstile_160109.txt > mta_data_jan09.csv

In [14]:
import csv

def mta_csv_to_dict(path_to_csv):
    with open(path_to_csv) as csvfile:
        mta_reader = csv.reader(csvfile, delimiter=',')
        mta_rows = [[cell.strip() for cell in row] for row in mta_reader]
    assert mta_rows.pop(0) == ['C/A','UNIT','SCP','STATION','LINENAME','DIVISION',
                               'DATE','TIME','DESC','ENTRIES','EXITS']
    mta_dict = {}
    for row in mta_rows:
        key, val =  tuple(row[:4]), row[4:]
        mta_dict.setdefault(key,[]).append(val)
    return mta_dict

mta_dict = mta_csv_to_dict('mta_data_jan09.csv')

# Challenge 2

Let's turn this into a time series. For each key (basically the control area, unit, device address and station of a specific turnstile), have a list again, but let the list be comprised of just the point in time and the count of entries. This basically means keeping only the date, time, and entries field in each list.

In [ ]:
import dateutil.parser

mta_cummulative_timeseries = {
    
    
}